# Advent of Code 2022

## Day 24: Blizzard Basin

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

I pulled up the Dijkstra search I wrote for day 12 and modified it to do A*.

I created a `ValleyMap` object which has *n* `ValleyMapState` objects where *n* is the period of the map. The period of the map is the LCM(width, height) where width, height are the interior dimension (so exclude the band of 1 around the outside). This allows precomputing location of walkable ground at every timestep in constant time lookup after all *n* states have been precomputed. n = lcm(6, 4) = 12 for the example and n = lcm(35, 100) = 700 for the real input file.



### Helpers

I wrote generic Dijkstra/A* stuff and put it in a file `day24/search.py` to make this notebook a little tidier.

In [ ]:
from day24.search import a_star, ArrayHeap, manhattan

### Imports

In [ ]:
import numpy as np
from math import lcm
from tqdm.autonotebook import tqdm
from typing import Iterator, Callable

### The Valley

In [ ]:
WALL = -10000
UP = 1
RIGHT = 2
DOWN = 4
LEFT = 8
DECODING = {'.' : 0,
            '#' : WALL,
            '^' : UP, '>' : RIGHT, 'v' : DOWN, '<' : LEFT}
HORIZONTAL = {DECODING['<'], DECODING['>']}
VERTICAL = {DECODING['^'], DECODING['v']}
EMOJI = {
    0 : '⬛',      # empty space
    WALL: '◻️',    # wall
    UP: '⬆️',      # 0001
    RIGHT: '➡️️',   # 0010
    3: '2️⃣',   # 0011
    DOWN: '⬇️',   # 0100
    5: '2️⃣',   # 0101
    6: '2️⃣',   # 0110
    7: '3️⃣',   # 0111
    LEFT: '⬅️',   # 1000
    9: '2️⃣',   # 1001
    10: '2️⃣',  # 1010
    11: '3️⃣',  # 1011
    12: '2️⃣',  # 1100
    13: '3️⃣',  # 1101
    14: '3️⃣',  # 1110
    15: '4️⃣'   # 1111
}

class ValleyMapState:

    __slots__ = ['arr']
    total_states_created = 0

    def __init__(self, arr: np.ndarray):
        self.arr = arr
        ValleyMapState.total_states_created += 1

    def is_ground(self, loc: tuple[int, int]) -> bool:
        height, width = self.arr.shape
        y, x = loc
        return (0 <= y < height) and (0 <= x <= width) and (self.arr[loc] == 0)

    def next_state(self) -> 'ValleyMapState':
        next_arr = np.zeros(self.arr.shape, dtype=int)
        height, width = self.arr.shape

        # copy walls
        for y in range(height):
            for x in range(width):
                if self.arr[y, x] == WALL:
                    next_arr[y, x] = WALL

        # looping over only 1 to n-2, so we check the interior only
        for y in range(1, height - 1):
            y_above = (((y - 1) - 1) % (height - 2)) + 1
            y_below = (((y - 1) + 1) % (height - 2)) + 1
            assert 1 <= y_above < (height - 1)
            assert 1 <= y_below < (height - 1)
            for x in range(1, width - 1):
                loc = y, x
                x_left = (((x - 1) - 1) % (width - 2)) + 1
                x_right = (((x - 1) + 1) % (width - 2)) + 1
                assert 1 <= x_left < (width - 1)
                assert 1 <= x_right < (width - 1)
                above = y_above, x
                right = y, x_right
                below = y_below, x
                left = y, x_left
                assert (self.arr[above] >= 0), f'{self.arr[above] = }, where {(x, y) = }, {above = }, expected >= 0'
                assert (self.arr[right] >= 0), f'{self.arr[right] = }, where {(x, y) = }, {right = }, expected >= 0'
                assert (self.arr[below] >= 0), f'{self.arr[below] = }, where {(x, y) = }, {below = }, expected >= 0'
                assert (self.arr[left] >= 0), f'{self.arr[left] = }, where {(x, y) = }, {left = }, expected >= 0'
                num = 0
                if (self.arr[right] & LEFT) == LEFT:
                    next_arr[loc] += LEFT
                    num += 1
                if (self.arr[below] & UP) == UP:
                    next_arr[loc] += UP
                    num += 1
                if (self.arr[above] & DOWN) == DOWN:
                    next_arr[loc] += DOWN
                    num += 1
                if (self.arr[left] & RIGHT) == RIGHT:
                    next_arr[loc] += RIGHT
                    num += 1
                assert 0 <= num <= 4

        return ValleyMapState(next_arr)

    def emoji(self, loc: tuple[int, int]) -> str:
        return EMOJI[self.arr[loc]]

    def __repr__(self):
        return str(self)

    def __str__(self):
        rv = ''
        height, width = self.arr.shape
        for y in range(height):
            for x in range(width):
                rv += self.emoji((y, x))
            rv += '\n'
        return rv

ValleyMapState.total_states_created = 0

def spatial_neighbours(point: tuple[int, int]) -> Iterator[tuple[int, int]]:
    y, x = point
    yield y, x
    yield y-1, x
    yield y, x+1
    yield y+1, x
    yield y, x-1


class ValleyMap:

    __slots__ = ['height', 'width', 'states', 'horizontal_period', 'vertical_period', 'period', 'start', 'end', 'heuristic']

    def __init__(self, arr: np.ndarray):
        self.height, self.width = arr.shape
        self.start = (0, 1)
        self.end = (self.height - 1, self.width - 2)
        self.horizontal_period = self.width - 2
        self.vertical_period = self.height - 2
        self.period = lcm(self.horizontal_period, self.vertical_period)
        for y in range(self.height):
            x = self.start[1]
            assert arr[y, x] not in VERTICAL
        for y in range(self.height):
            x = self.end[1]
            assert arr[y, x] not in VERTICAL
        self.states = [None] * self.period
        self.states[0] = ValleyMapState(arr)
        def heuristic(vertex: tuple[int, tuple[int, int]]) -> int:
            _, point = vertex
            return manhattan(point, self.end)
        self.heuristic: Callable[[tuple[int, tuple[int, int]]], int] = heuristic

    def neighbours(self, vertex: tuple[int, tuple[int, int]]) -> tuple[tuple[int, tuple[int, int]], int]:
        time, point = vertex
        for n in spatial_neighbours(point):
            if self.is_ground(time + 1, n):
                yield (time + 1, n), 1

    def is_goal(self, vertex: tuple[int, tuple[int, int]]) -> bool:
        _, point = vertex
        return point == self.end

    def is_ground(self, time: int, loc: tuple[int, int]) -> bool:
        return self[time].is_ground(loc)

    def precompute_states(self, pbar: bool = False) -> None:
        if pbar:
            for i in tqdm(range(self.period), desc='Precomputing Valley States'):
                _ = self[i]  # called for side effect
        else:
            for i in range(self.period):
                _ = self[i]  # called for side effect
        assert None not in self.states

    def __getitem__(self, item: int) -> ValleyMapState:
        assert type(item) == int and item >= 0
        item = item % len(self.states)
        if self.states[item] is None:
            self.states[item] = self[item - 1].next_state()  # recursive call
        return self.states[item]

    def __repr__(self):
        return str(self)

    def __str__(self):
        return str(self[0])

    @staticmethod
    def read(filename: str) -> 'ValleyMap':
        lines = []
        with open(filename) as file:
            for line in file:
                lines.append([DECODING[char] for char in line.strip()])
        rv = np.array(lines)
        return ValleyMap(rv)

### Part 1

In [ ]:
def main():

    # load the valley and precompute states
    valley_map = ValleyMap.read(INPUT_FILE)
    valley_map.precompute_states(pbar=(valley_map.period > 12))
    print('Precomputed states')

    # start state is valley start point a time 0
    start = 0, valley_map.start

    path, visited = a_star(ArrayHeap,
                           start,
                           valley_map.neighbours,
                           valley_map.is_goal,
                           valley_map.heuristic,
                           pbar=(valley_map.period > 12))

    end_time, end_point = path[-1]
    assert end_point == valley_map.end

    print(f'Shortest path to {end_point} is after {end_time} time steps')

In [ ]:
INPUT_FILE = 'data/example24.txt'
#INPUT_FILE = 'data/input24.txt'

if __name__ == '__main__':
    main()